In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
class LogisticRegression:
	def __init__(self, learning_rate = 0.01, epoch = 2000):
		self.learning_rate = learning_rate
		self.epoch = epoch
		self.w = []
		self.b = 0

	def initialize_weigth(self, dim):
		w = np.random.normal(0, 1, (dim, 1))
		b = np.random.rand(1)
		return w, b

	def sigmoid(self, x):
		return 1 / (1 + np.exp(-x))

	def hypothesis(self, w, X, b):
		y_hat = self.sigmoid(np.matmul(X, w) + b)
		y_hat = np.squeeze(y_hat)
		return y_hat
	
	def cost(self, y_hat, y, N):
		cost = -(1 / N) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
		cost = np.squeeze(cost)
		return cost

	def cal_gradient(self, w, y_hat, X, y):
		N = X.shape[0]
		delta_w = (1 / N) * np.matmul(X.T, (y_hat - y))
		delta_b = (1 / N) * np.sum(y_hat - y)
		grads = {
			"delta_w": delta_w,
			"delta_b": delta_b
		}
		return grads

	def gradient_position(self, w, b, X, y):
		N = X.shape[0]
		y_hat = self.hypothesis(w, X, b)
		cost = self.cost(y_hat, y, N)
		grads = self.cal_gradient(w, y_hat, X, y)
		return grads, cost

	def gradient_descent(self, w, b, X, y, print_cost = False):
		costs = []
		for i in range(self.epoch):
			grads, cost = self.gradient_position(w, b, X, y)

			delta_w = grads["delta_w"]
			delta_b = grads["delta_b"]

			delta_w = delta_w.reshape(-1, 1)

			w = w - (self.learning_rate * delta_w)
			b = b - (self.learning_rate * delta_b)
			if i % 100 == 0:
				costs.append(cost)

			if print_cost and i % 100 == 0:
				print("Cost after iteration %i: %f" %(i, cost))
			
			params = {
				"w": w,
				"b": b
			}
			grads = {
				"delta_w": delta_w,
				"delta_b": delta_b
			}

		return params, costs

	def predict(self, X):
		X = np.array(X)
		N = X.shape[0]

		Y_prediction = np.zeros(N)

		w = self.w.reshape(X.shape[1], 1)
		b = self.b

		y_hat = self.hypothesis(w, X, b)
		for i in range(len(y_hat)):
			if y_hat[i] >= 0.5:
				Y_prediction[i] = 1
			else:
				Y_prediction[i] = 0
		
		return Y_prediction

	def train_model(self, X_train, Y_train, X_test, Y_test, print_cost = False):
		dim = np.shape(X_train)[1]
		w, b = self.initialize_weigth(dim)
		parameters, costs = self.gradient_descent(w, b, X_train, Y_train, print_cost=False)

		self.w = parameters['w']
		self.b = parameters['b']

		Y_prediction_test = self.predict(X_test)
		Y_prediction_train = self.predict(X_train)

		train_score = 100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100
		test_scroe = 100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100
		print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

		result_dict = {
			"costs": costs,
			"Y_prediction_test": Y_prediction_test,
			"Y_prediction_train": Y_prediction_train,
			"w": self.w,
			"b": self.b,
			"learning_rate": self.learning_rate,
			"num_iteration": self.epoch,
			"train_accuracy": train_score,
			"test_accuracy": test_scroe
		}

		return result_dict

In [ ]:
xy_data = np.load('./data.npz')

X = xy_data['x']
Y = xy_data['y']

train_number = int(Y.shape[0] * 0.9)

X_train = X[:train_number, :]
Y_train = Y[:train_number]

X_test = X[train_number:, :]
Y_test = Y[train_number:]

plt.scatter(X_train[Y_train == 0][:, 0],
            X_train[Y_train == 0][:, 1], color='red')
plt.scatter(X_train[Y_train == 1][:, 0],
            X_train[Y_train == 1][:, 1], color='green')
plt.show()
plt.close()


In [ ]:
LR = LogisticRegression()
result_dict = LR.train_model(X_train, Y_train, X_test, Y_test)